In [22]:
import functools

import numpy as np
import tensorflow as tf

In [23]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test.csv", TEST_DATA_URL)

In [24]:
np.set_printoptions(precision=3, suppress=True)

In [25]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [26]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [27]:
examples, lables = next(iter(raw_train_data))
print("Example: \n", examples, "\n")
print("LABELS:\n", lables)

Example: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'male', b'female', b'female', b'female', b'female',
       b'female', b'male', b'female', b'female', b'male', b'female'],
      dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([32., 30., 35., 45.,  4., 27., 25., 40., 23., 24., 21., 18.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 1, 0, 1, 0, 1, 0, 3, 0, 0, 0], dtype=int32)>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1], dtype=int32)>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([  8.05 ,  13.   ,  83.475,   7.75 ,  23.   ,   7.925,  30.   ,
         0.   , 263.   ,  13.   ,   7.775,   9.35 ], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'Third', b'Second', b'First', b'Third', b'Second', b'Third',
       b'Second', b'First', b'First', b'Second', b

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class': ['First', 'Second', 'Third'],
    'deck': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town': ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone': ['y', 'n']
}

In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab
    )
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
def process_continuous_data(mean, data):
    data = tf.cast(data, tf.float32) * 1 / (2 * mean)
    return tf.reshape(data, [-1, 1])

In [32]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []
for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [33]:
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x145c9e3b0>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x145c9e3b0>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x145c9e3b0>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x145c9e3b0>, 34.385399))]

In [34]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numerical_columns)

In [35]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [36]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

model.fit(train_data, epochs=20)

Epoch 1/20
53/53 [==============================] - 0s 1ms/step - loss: 0.5219 - accuracy: 0.7560
Epoch 2/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4354 - accuracy: 0.8134
Epoch 3/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8086
Epoch 4/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4088 - accuracy: 0.8182
Epoch 5/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4036 - accuracy: 0.8262
Epoch 6/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3964 - accuracy: 0.8262
Epoch 7/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3911 - accuracy: 0.8246
Epoch 8/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3862 - accuracy: 0.8230
Epoch 9/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3778 - accuracy: 0.8405
Epoch 10/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3733 - accuracy: 0.8293
Epoch 11/20
53/53 [

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

22/22 [==============================] - 0s 2ms/step - loss: 0.4763 - accuracy: 0.7917


In [38]:
predictions = model.predict(test_data)

for prediction, suvived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted surival: {:.2%}".format(prediction[0]),
    "| Actual outcome: ",
    ("SURVIED" if bool(suvived) else "DIED")
    )

Predicted surival: 1.87% | Actual outcome:  SURVIED
Predicted surival: 10.53% | Actual outcome:  DIED
Predicted surival: 38.92% | Actual outcome:  SURVIED
Predicted surival: 92.27% | Actual outcome:  SURVIED
Predicted surival: 98.46% | Actual outcome:  DIED
Predicted surival: 72.83% | Actual outcome:  DIED
Predicted surival: 35.00% | Actual outcome:  SURVIED
Predicted surival: 91.27% | Actual outcome:  DIED
Predicted surival: 8.04% | Actual outcome:  SURVIED
Predicted surival: 37.38% | Actual outcome:  SURVIED
